In [2]:
import xtrack as xt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import apertls

**MBB**
\
*rect aperture*:
50.14 mm - 126.12 mm
\
*circle*:
103.00 mm
\
**VCAK**
*big circle*:
103.00 mm
\
*small circle*:
43.00 mm
\
*limit aperture*
r = 41.5mm, displacement of center of 5.3mm to the left

In [4]:
path_lines = Path('/Users/gebruiker/sps_simulations/injection_lines')
line = xt.Line.from_json(Path(path_lines, 'sps_with_aperture_inj_q20.json'))

Loading line from dict:   0%|          | 0/17781 [00:00<?, ?it/s]

Done loading line from dict.           


In [ ]:
tt = line.get_table()
tt.rows['mbb.*']

Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/gebruiker/miniforge3/envs/xsuite2/lib' ignored
ld: warning: duplicate -rpath '/Users/gebruiker/miniforge3/envs/xsuite2/lib' ignored


Done compiling ContextCpu kernels.


In [16]:
x_aper_vebb = 0.0645*2 #width of rectangular aperture
x_aper_vebb_in = 92.5e-3 #radius
x_aper_vebb_out = 79.82*2e-3 #radius
x_aper_vebb_contour = 103*2e-3 #radius of output link with vcad
x_aper_vcad = 0.083 #diameter
x_aper_vcad_middle = 102e-3 #diameter of larger part bellow

l_mec_mbb = 6.2
l_magn_mbb = 6.260018602780449
bending_radius = 1/0.0013490601351007177

offset_middle_mbb = 2.2e-3
offset_end_mbb = -4.4e-3
offset_nom = -5.3e-3

l_vebb = 6.6410
l_vebb_in = 109e-3
l_vebb_out = 119e-3
l_vcad = 296e-3#1.422 #should be 296.6e-3? yes ...
l_vcad1 = 21e-3
l_vcad_middle = 70e-3
l_vcad_tube = 179e-3
l_vcad_link = 13e-3  


In [13]:
def circle(radius, x_0, y_0, t):
    x = x_0 + radius*np.cos(t)
    y = y_0 + radius*np.sin(t)

    return x, y

def find_y_circle(radius, x_0, y_0, x):
    t = np.acos((x-x_0)/radius)
    y = y_0 + radius*np.sin(t)

    return y

def find_curvature_offset():
    pass
def find_y_sagitta_circle(radius, x_0, y_0, x):
    return find_y_circle(radius, x_0, y_0, x) + find_curvature_offset()
    
def droite(alpha, x_0, y_0, x):
    return alpha*(x-x_0) + y_0

In [17]:
nb_bends = len(tt.rows['mb.*'].name)
angle_per_bend = 2*np.pi/nb_bends
s_end_mbb, x_end_mbb = circle(bending_radius, 0, 0, np.pi/2 - angle_per_bend/2)
subtracted_circle_part = find_y_circle(1/0.0013490601351007177, 0,0, l_magn_mbb/2) - offset_end_mbb
x_end_mbb -= subtracted_circle_part

s_arr_mbb = np.linspace(-s_end_mbb, s_end_mbb, 100)
circle_points = find_y_circle(bending_radius, 0, 0, s_arr_mbb) - subtracted_circle_part

s_arr_vebb1 = np.linspace(-l_vebb/2, -s_end_mbb, 10)
s_arr_vebb2 = np.linspace(s_end_mbb, l_vebb/2, 10)
h = (s_arr_mbb[-1]-s_arr_mbb[0])/100
der0 = (circle_points[1]-circle_points[0])/h

s_arr_vcad = np.linspace(l_vebb/2, l_vebb/2 + l_vcad, 10)

NameError: name 'tt' is not defined

In [18]:
%matplotlib tk
fig, ax = plt.subplots(1,1)

#reference orbit in MBB with sagitta
ax.plot(s_arr_mbb, circle_points, '--', color='black')
ax.scatter([-s_end_mbb, s_end_mbb], [x_end_mbb, x_end_mbb], color='black')
ax.plot(s_arr_vebb1, droite(der0, -s_end_mbb, x_end_mbb, s_arr_vebb1), '--', color='black')
ax.plot(s_arr_vebb2, droite(-der0, s_end_mbb, x_end_mbb, s_arr_vebb2), '--', color='black')
ax.plot(s_arr_vcad, droite(-der0, s_end_mbb, x_end_mbb, s_arr_vcad), '--', color='black')
# ax.axis('equal')

#MBB and VEBB patches
mbb_up_patch = patches.Rectangle(xy=(-l_mec_mbb/2, x_aper_vebb/2), width=l_mec_mbb, height=0.2, color='red', alpha=0.7)
mbb_down_patch = patches.Rectangle(xy=(-l_mec_mbb/2, -x_aper_vebb/2), width=l_mec_mbb, height=-0.2, color='red', alpha=0.7)
vebb_patch = patches.Rectangle(xy=(-l_vebb/2, -x_aper_vebb/2), width=l_vebb, height=x_aper_vebb, color='green', alpha=0.3)
vebb_patch_in = patches.Rectangle(xy=(-l_vebb/2 - l_vebb_in, -x_aper_vebb_in), width=l_vebb_in, height=2*x_aper_vebb_in, color='green', alpha=0.3)
vebb_patch_out = patches.Rectangle(xy=(l_vebb/2, -x_aper_vebb_out), width=l_vebb_out, height=2*x_aper_vebb_out, color='green', alpha=0.3)
vcad_patch_in = patches.Rectangle(xy=(l_vebb/2 + l_vebb_out, -x_aper_vcad/2), width=l_vcad_link + l_vcad1, height=x_aper_vcad, color='tab:blue', alpha=0.3)
vcad_patch_middle = patches.Rectangle(xy=(l_vebb/2 + l_vebb_out + l_vcad_link + l_vcad1, -x_aper_vcad_middle/2 - np.abs(offset_nom)), width=l_vcad_middle, height=x_aper_vcad_middle, angle=angle_per_bend/2, color='tab:blue', alpha=0.3)
vcad_patch_tube = patches.Rectangle(xy=(l_vebb/2 + l_vebb_out + l_vcad_link + l_vcad1 +l_vcad_middle, -x_aper_vcad/2 - np.abs(offset_nom)), width=l_vcad_tube, height=x_aper_vcad, angle=angle_per_bend/2, color='tab:blue', alpha=0.3)
ax.add_patch(mbb_up_patch)
ax.add_patch(mbb_down_patch)
ax.add_patch(vebb_patch)
ax.add_patch(vebb_patch_in)
ax.add_patch(vebb_patch_out)
ax.add_patch(vcad_patch_in)
ax.add_patch(vcad_patch_middle)
ax.add_patch(vcad_patch_tube)

ax.plot(s_arr_vcad, droite(-der0, s_end_mbb, offset_end_mbb, s_arr_vcad), '--', color='tab:orange')
# ax.hlines(offset_nom, l_vebb/2, l_vebb/2 + l_vcad, color='tab:orange', linestyles='--')